# Efficiency - Add a single point

| comparable algo list |
| - |
| pivot |
| delta |
| Monte Carlo |

In [ ]:
import sys
sys.path.append("..")

from sklearn import svm
import numpy as np

import dynashap

from examples.data_utils import (
    load_npy, save_npy, preprocess_data, variance, normalize
)

In [ ]:
model = svm.SVC(decision_function_shape='ovo')

size_choices = [10, 100, 1000, 10000]
m_choices = [10, 15, 18, 20, 22, 25, 28, 30]

stone_m = 100

for size in size_choices:
    x_train, y_train, x_test, y_test, \
    columns_name = preprocess_data(
        't_train_' + str(size) + 'p.csv',
        't_test_' + str(size) + 'p.csv')

    # process data
    x_train_ = x_train[:-1]
    y_train_ = y_train[:-1]

    # for add
    add_point_x = x_train[-1]
    add_point_y = y_train[-1]

    x_test_ = x_test
    y_test_ = y_test

    # get a init sv
    init_sv = dynashap.mc_shap(x_train_, y_train_, x_test_, y_test_, model, stone_m * len(y_train_), proc_num=2)
    save_npy('as_init_mc_plus_size' + str(size) + '.npy', init_sv)

    # set a stone
    mc_plus = dynashap.mc_shap(x_train, y_train, x_test_, y_test_, model, stone_m * len(y_train), proc_num=2)
    save_npy('as_mc_plus_size' + str(size) + '.npy', mc_plus)

    # left sv for pivot
    pivot_shap = dynashap.PivotShap(x_train_, y_train_, x_test_, y_test_, model,
                                   None)
    lsv = pivot_shap.prepare(m=len(y_train_)*stone_m)
    save_npy('as_pivot_lsv_size' + str(size) + '.npy', lsv)

    # mc
    mc = dynashap.mc_shap(x_train, y_train, x_test_, y_test_, model, mc_stone * len(y_train))
    save_npy('as_mc'+ str(size) + 'm' + str(mc_stone) + '.npy', mc)

    # # walk the m choices
    for m in m_choices:

        # delta
        delta_shap = dynashap.DeltaShap(x_train_, y_train_, x_test_, y_test_, model,
                                        init_sv)
        delta_sv = delta_shap.add_single_point(add_point_x,
                                               add_point_y,
                                               m=(len(y_train_)+1)*m)
        save_npy('as_delta_size' + str(size) +'m' + str(m) + '.npy', delta_sv)

        # pivot
        pivot_shap = dynashap.PivotShap(x_train_, y_train_, x_test_, y_test_, model,
                                        None)
        pivot_shap.left_sv = lsv
        pivot_sv = pivot_shap.add_single_point(add_point_x, add_point_y,
                                               m=(len(y_train_)+1)*m,
                                               {'flag_lsv': True})
        save_npy('as_pivot_size' + str(size) +'m' + str(m) + '.npy', pivot_sv)